objective 1: organize and query DSD files coming into email

objective 2: generate a database that describes all orders based on location with key details provided

objective 3: send an email (or generate an email to be sent) with the latest delivery details

objective 1: organize and query DSD files coming into email

1. connect to imaplib to query emails
2. filter out non-dsd emails
3. retrieve pdfs and read data
4. return: map to database and indicate as to-send (consider assigning invoices as well), log pdfs to a repository

In [93]:
import imaplib
import time
import base64
import os

ssl_host = 'imap.exmail.qq.com'
ssl_port = 993
username = 'intcocanada@intco.com'
password = 'IntCan24'
start = time.time()

try:
    imap = imaplib.IMAP4_SSL(host = ssl_host, port = ssl_port, timeout=2)
except Exception as e:
    print('ErrorType: {}, Error: {},'.format(type(e).__name__, e))
    imap = None

print('Connection Object: {}'.format(imap))
print('Total Time Taken: {:,.2f} Seconds'.format(time.time() - start))

Connection Object: <imaplib.IMAP4_SSL object at 0x00000211EBE0D880>
Total Time Taken: 1.29 Seconds


In [94]:
print('Logging into mailbox...')

try:
    resp_code, response = imap.login(username, password)
except Exception as e:
    print('ErrorType: {}, Error: {},'.format(type(e).__name__, e))
    resp_code, response = None

print("Response Code : {}".format(resp_code))
print("Response      : {}\n".format(response[0].decode()))


Logging into mailbox...
Response Code : OK
Response      : Success login ok



In [ ]:
print('Logging out of mailbox...')

try:
    resp_code, response = imap.logout()
except Exception as e:
    print('ErrorType: {}, Error: {},'.format(type(e).__name__, e))
    resp_code, response = None

print("Response Code : {}".format(resp_code))
print("Response      : {}\n".format(response[0].decode()))
    

In [ ]:
try:
    resp_code, directories = imap.list()
except Exception as e:
    print("ErrorType : {}, Error : {}".format(type(e).__name__, e))
    resp_code, directories = None, None

print("Response Code : {}".format(resp_code))
print("========= List of Directories =================\n")
for directory in directories:
    print(directory.decode())

In [125]:
imap.select('INBOX')

('OK', [b'284'])

In [126]:
criteria = '(SUBJECT "Purchase Order" FROM "tntpo@tntsupermarket.com" <tntpo@tntsupermarket.com>)'
status, data = imap.search(None, criteria)

In [138]:
message_ids = data[0].decode().split()
message_ids[247]
# message = email.message_from_bytes(imap.fetch('248', '(RFC822)'))
message = imap.fetch('247', '(RFC822)')
for part in email.message_from_bytes(message[1][0][1]).walk():
    if part.get_content_maintype() == 'multipart':
        continue
    if part.get('Content-Disposition') is None:
        continue
    fileName = part.get_filename()
    if bool(fileName):
        filePath = os.path.join(r"C:\Users\Owner\Downloads", fileName)
        if not os.path.isfile(filePath) :
            fp = open(filePath, 'wb')
            fp.write(part.get_payload(decode=True))
            fp.close()

In [ ]:
message_ids = data[0].decode().split()
message_ids
for id in reversed(message_ids):
    message = email.message_from_bytes(imap.fetch(id, '(RFC822)')[1][0][1])
    if message.get('From') == '"tntpo@tntsupermarket.com" <tntpo@tntsupermarket.com>':
        print('{}, Date: {}, {}: {}'.format(id, message.get('Date'), message.get('From'), message.get('Subject')))
    
    # for part in message.walk():
    #     if part.get_content_type() in ['text/plain']:
    #         body_lines = part.as_string().split('\n')
    #         body = base64.b64decode("".join(body_lines[4:]))
    #         body = body.decode('utf-8')
    #         print(email.message_from_string(body))

In [ ]:
resp_code, inbox = imap.select('INBOX', readonly=True)
resp_code, mail_ids = imap.search(None, 'FROM ")')

In [ ]:
import email
import base64
import os
for mail_id in mail_ids[0].decode().split()[-2]:
    resp_code, mail_data = imap.fetch(mail_id, '(RFC822)')
    message = email.message_from_bytes(mail_data[0][1])
    print('Mail Id: {}'.format(mail_id))
    print('From: {}'.format(message.get('From')))
    for part in message.walk():
        if part.get_content_type() == 'text/plain':
            body_lines = part.as_string().split('\n')
            print('\n'.join(body_lines[:12]))
    break

In [ ]:
message = email.message_from_bytes(imap.fetch(mail_ids[0].decode().split()[-1], '(RFC822)')[1][0][1])
for part in message.walk():
    if part.get_content_type() in ['text/plain']:
        body_lines = part.as_string().split('\n')
        body = base64.b64decode("".join(body_lines[4:]))
        # display(email.message_from_string(body))
        body = body.decode('utf-8')
        
    # display(body_lines[7].split('\n'))
    # display('break')

In [ ]:
print(email.message_from_string(body))

In [ ]:
mail_ids = data[0]
mail_ids = mail_ids.split()
latest_email = mail_ids[-1]
result, data  = imap.fetch(latest_email, '(RFC822)')
raw_email = data[0][1]
raw_email.get('FROM')

objective 2: generate a database that describes all orders based on location with key details provided

Summary: Build database based on key variables
1) Subject Line - email
2) PO Number - pdf
3) Order Date - pdf
4) Delivery Date - pdf
5) Store - pdf
6) Address - pdf
7) Item, Description, Quantity, Price - pdf
8) delivery status - logic

objective 3: send an email (or generate an email to be sent) with the latest delivery details

1. query database for deliveries not completed yet
2. prepare email body
3. retrieve POs and Invoices to attach to email body